# batch

> Fill in a module description here

In [ ]:
#| default_exp batch

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import time
import logging
import threading
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
from dataclasses import asdict
from concurrent.futures import ThreadPoolExecutor, as_completed

from llm_data_extractor.models import Question, LLMConfig, AnswerType
from llm_data_extractor.extract import extract_data
from llm_data_extractor.snowflake_client import (
    SnowflakeConfig, insert_extraction_results, format_results_for_snowflake,
    get_pending_extractions
)


In [ ]:
#| export

logger = logging.getLogger(__name__)


class BatchProcessor:
    """Main orchestrator for batch processing LLM extractions with Snowflake using workers."""
    
    def __init__(self, snowflake_config: SnowflakeConfig, llm_config: LLMConfig, max_workers: int = 4):
        self.snowflake_config = snowflake_config
        self.llm_config = llm_config
        self.max_workers = max_workers
        self._insert_lock = threading.Lock()  # For thread-safe database inserts
        
    def process_from_snowflake(self, 
                              query: str,
                              query_params: Optional[Dict[str, Any]] = None,
                              max_batches: Optional[int] = None,
                              dry_run: bool = False) -> Dict[str, Any]:
        """
        Process pending extractions by running a query against Snowflake.
        
        Args:
            query: SQL query to get pending extraction records
            query_params: Optional parameters for the query
            max_batches: Maximum number of input batches to process
            dry_run: If True, don't insert results back to database
            
        Returns:
            Dictionary with processing summary and statistics
            
        Expected query result columns:
            - INPUT_ROW_ID: Identifier for the input record
            - INPUT_TEXT: Text to extract data from  
            - QUESTION_ID: Question identifier
            - QUESTION_TEXT: The question text
            - ANSWER_TYPE: Type of expected answer
            - ANSWER_CONFIG: JSON configuration for the answer
        """
        
        start_time = time.time()
        
        try:
            # Get data from Snowflake using user's query
            logger.info("Fetching pending extractions from Snowflake...")
            pending_records = get_pending_extractions(
                snowflake_config=self.snowflake_config,
                query=query,
                params=query_params
            )
            
            if not pending_records:
                logger.info("No pending extractions found")
                return self._empty_result(start_time)
            
            # Convert to DataFrame and use existing processing logic
            df = pd.DataFrame(pending_records)
            
            # Ensure column names are uppercase (Snowflake default)
            df.columns = [col.upper() for col in df.columns]
            
            # Validate required columns
            required_columns = ['INPUT_ROW_ID', 'INPUT_TEXT', 'QUESTION_ID', 'QUESTION_TEXT', 'ANSWER_TYPE', 'ANSWER_CONFIG']
            missing_columns = [col for col in required_columns if col not in df.columns]
            
            if missing_columns:
                raise ValueError(f"Query result missing required columns: {missing_columns}")
            
            logger.info(f"Retrieved {len(df)} pending extraction records from Snowflake")
            
            # Use existing dataframe processing logic
            return self.process_dataframe(df, max_batches, dry_run)
            
        except Exception as e:
            logger.error(f"Failed to process from Snowflake: {str(e)}")
            return {
                'status': 'failed',
                'error': str(e),
                'processing_time_seconds': round(time.time() - start_time, 2)
            }
    
    def process_dataframe(self, 
                         df: pd.DataFrame,
                         max_batches: Optional[int] = None,
                         dry_run: bool = False) -> Dict[str, Any]:
        """
        Process pending extractions from a provided dataframe using concurrent workers.
        
        Args:
            df: DataFrame with pending extraction records (from your SQL query)
            max_batches: Maximum number of input batches to process
            dry_run: If True, don't insert results back to database
            
        Returns:
            Dictionary with processing summary and statistics
        """
        
        start_time = time.time()
        
        try:
            if df.empty:
                logger.info("No pending extractions in provided dataframe")
                return self._empty_result(start_time)
            
            logger.info(f"Processing {len(df)} pending extraction records with {self.max_workers} workers")
            
            # Step 1: Group records into batches by input_row_id
            batches = self._batch_dataframe_records(df)
            
            # Limit batches if requested
            if max_batches:
                batch_keys = list(batches.keys())[:max_batches]
                batches = {k: batches[k] for k in batch_keys}
                logger.info(f"Limited to {max_batches} batches")
            
            # Step 2: Process batches concurrently
            processing_stats = {
                'batches_processed': 0,
                'batches_succeeded': 0,
                'batches_failed': 0,
                'total_questions': 0,
                'total_valid_answers': 0,
                'total_invalid_answers': 0,
                'total_inserted': 0,
                'errors': []
            }
            
            # Use ThreadPoolExecutor for concurrent processing
            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                # Submit all batch processing tasks
                future_to_batch = {
                    executor.submit(self._process_and_insert_batch, batch_data, dry_run): batch_key 
                    for batch_key, batch_data in batches.items()
                }
                
                # Process completed tasks as they finish
                for future in as_completed(future_to_batch):
                    batch_key = future_to_batch[future]
                    
                    try:
                        batch_result = future.result()
                        
                        # Update stats thread-safely
                        with self._insert_lock:
                            processing_stats['batches_processed'] += 1
                            processing_stats['batches_succeeded'] += 1
                            processing_stats['total_questions'] += batch_result['stats']['total_questions']
                            processing_stats['total_valid_answers'] += batch_result['stats']['valid_answers']
                            processing_stats['total_invalid_answers'] += batch_result['stats']['invalid_answers']
                            processing_stats['total_inserted'] += batch_result['rows_inserted']
                        
                        logger.info(f"✓ Batch {batch_key}: {batch_result['stats']['success_rate']:.1%} success, "
                                  f"{batch_result['rows_inserted']} rows inserted")
                        
                    except Exception as e:
                        with self._insert_lock:
                            processing_stats['batches_failed'] += 1
                            processing_stats['errors'].append(f"Batch {batch_key}: {str(e)}")
                        logger.error(f"✗ Batch {batch_key} failed: {str(e)}")
            
            # Calculate overall success rate
            total_questions = processing_stats['total_valid_answers'] + processing_stats['total_invalid_answers']
            overall_success_rate = (processing_stats['total_valid_answers'] / total_questions) if total_questions > 0 else 0
            
            # Final summary
            summary = {
                'status': 'completed' if not processing_stats['errors'] else 'completed_with_errors',
                'total_pending_records': len(df),
                'unique_batches': len(batches),
                'batches_processed': processing_stats['batches_processed'],
                'batches_succeeded': processing_stats['batches_succeeded'],
                'batches_failed': processing_stats['batches_failed'],
                'total_questions': processing_stats['total_questions'],
                'total_valid_answers': processing_stats['total_valid_answers'],
                'total_invalid_answers': processing_stats['total_invalid_answers'],
                'overall_success_rate': round(overall_success_rate, 3),
                'results_inserted': processing_stats['total_inserted'],
                'processing_time_seconds': round(time.time() - start_time, 2),
                'avg_batch_time': round((time.time() - start_time) / len(batches), 2) if batches else 0,
                'workers_used': self.max_workers,
                'errors': processing_stats['errors']
            }
            
            logger.info(f"🎉 Processing completed: {summary['overall_success_rate']:.1%} success rate, "
                       f"{summary['results_inserted']} records inserted, "
                       f"{summary['processing_time_seconds']:.1f}s total")
            
            return summary
            
        except Exception as e:
            logger.error(f"Batch processing failed: {str(e)}")
            return {
                'status': 'failed',
                'error': str(e),
                'processing_time_seconds': round(time.time() - start_time, 2)
            }
    
    def _process_and_insert_batch(self, batch_data: Dict[str, Any], dry_run: bool) -> Dict[str, Any]:
        """
        Process a single batch and immediately insert results.
        This method runs in a worker thread.
        
        Args:
            batch_data: Dictionary containing input_text and questions
            dry_run: If True, don't actually insert to database
            
        Returns:
            Dictionary with batch results and insertion count
        """
        
        batch_start_time = time.time()
        batch_id = batch_data['input_row_id']
        
        try:
            # Convert question dictionaries to Question objects
            questions = []
            for q_data in batch_data['questions']:
                try:
                    question = Question(
                        id=q_data['id'],
                        text=q_data['text'],
                        answer_type=AnswerType(q_data['answer_type']),
                        target_table=q_data['target_table'],
                        target_field=q_data['target_field'],
                        constraints=q_data.get('constraints', {})
                    )
                    questions.append(question)
                except Exception as e:
                    logger.error(f"Failed to create Question object for {q_data['id']}: {str(e)}")
                    continue
            
            if not questions:
                raise Exception("No valid questions found in batch")
            
            # Extract data using LLM
            logger.debug(f"Worker processing batch {batch_id} with {len(questions)} questions")
            extraction_results, metadata = extract_data(
                source_text=batch_data['input_text'],
                questions=questions,
                llm_config=self.llm_config,
                source_id=batch_id
            )
            
            # Format results for Snowflake
            snowflake_records = format_results_for_snowflake(
                extraction_results=extraction_results,
                input_row_id=batch_id,
                metadata=metadata
            )
            
            # Insert results immediately (thread-safe)
            rows_inserted = 0
            if not dry_run and snowflake_records:
                with self._insert_lock:
                    rows_inserted = insert_extraction_results(self.snowflake_config, snowflake_records)
            elif dry_run:
                rows_inserted = len(snowflake_records)  # For dry run reporting
            
            # Calculate statistics
            valid_count = sum(1 for r in extraction_results if r.is_valid)
            invalid_count = len(extraction_results) - valid_count
            success_rate = valid_count / len(extraction_results) if extraction_results else 0
            
            batch_time = time.time() - batch_start_time
            
            return {
                'batch_id': batch_id,
                'rows_inserted': rows_inserted,
                'processing_time': batch_time,
                'stats': {
                    'total_questions': len(extraction_results),
                    'valid_answers': valid_count,
                    'invalid_answers': invalid_count,
                    'success_rate': success_rate,
                    'avg_confidence': metadata.get('summary_stats', {}).get('avg_confidence', 0.0)
                },
                'metadata': metadata
            }
            
        except Exception as e:
            # Log the full error for this batch
            logger.error(f"Batch {batch_id} processing failed after {time.time() - batch_start_time:.1f}s: {str(e)}")
            raise e
    
    def _batch_dataframe_records(self, df: pd.DataFrame) -> Dict[str, Dict[str, Any]]:
        """
        Group dataframe records by input_row_id for batch processing.
        
        Args:
            df: DataFrame with pending extraction records
            
        Returns:
            Dictionary with input_row_id as key and batch info as value
        """
        
        batches = {}
        
        for _, record in df.iterrows():
            input_row_id = str(record['INPUT_ROW_ID'])
            
            if input_row_id not in batches:
                batches[input_row_id] = {
                    'input_row_id': input_row_id,
                    'input_text': record['INPUT_TEXT'],
                    'questions': [],
                    'question_configs': {}
                }
            
            # Parse answer_config JSON if it exists
            answer_config = {}
            if pd.notna(record['ANSWER_CONFIG']) and record['ANSWER_CONFIG']:
                try:
                    if isinstance(record['ANSWER_CONFIG'], dict):
                        answer_config = record['ANSWER_CONFIG']
                    else:
                        import json
                        answer_config = json.loads(record['ANSWER_CONFIG'])
                except (json.JSONDecodeError, TypeError):
                    logger.warning(f"Invalid ANSWER_CONFIG for question {record['QUESTION_ID']}")
                    answer_config = {}
            
            # Create question object
            question_data = {
                'id': str(record['QUESTION_ID']),
                'text': record['QUESTION_TEXT'],
                'answer_type': record['ANSWER_TYPE'],
                'target_table': 'processed_formstack_data',
                'target_field': 'message',
                'constraints': answer_config
            }
            
            batches[input_row_id]['questions'].append(question_data)
            batches[input_row_id]['question_configs'][str(record['QUESTION_ID'])] = record.to_dict()
        
        logger.info(f"Created {len(batches)} batches from {len(df)} pending records")
        return batches
    
    def _empty_result(self, start_time: float) -> Dict[str, Any]:
        """Return empty result structure."""
        return {
            'status': 'completed',
            'total_pending_records': 0,
            'unique_batches': 0,
            'batches_processed': 0,
            'batches_succeeded': 0,
            'batches_failed': 0,
            'results_inserted': 0,
            'processing_time_seconds': round(time.time() - start_time, 2)
        }

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()